## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-05-16-49-12 +0000,bbc,Lisbon funicular worker among those killed in ...,https://www.bbc.com/news/articles/cnvryg271ymo...
1,2025-09-05-16-48-54 +0000,nypost,DC US Attorney Jeanine Pirro charges 2 teens w...,https://nypost.com/2025/09/05/us-news/dc-us-at...
2,2025-09-05-16-48-02 +0000,startribune,Suspected Minneapolis carjacker charged with c...,https://www.startribune.com/carjacker-fleeing-...
3,2025-09-05-16-46-35 +0000,nypost,CBS News vows to air only uncut interviews on ...,https://nypost.com/2025/09/05/media/cbs-news-v...
4,2025-09-05-16-46-05 +0000,bbc,Rayner 'regrets' not seeking tax advice as she...,https://www.bbc.com/news/articles/c80gr5emk43o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2346/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
129,trump,60
304,will,13
87,up,13
440,department,13
139,market,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
243,2025-09-05-00-19-00 +0000,wsj,President Trump is expected to sign an executi...,https://www.wsj.com/politics/national-security...,130
237,2025-09-05-01-00-00 +0000,wsj,President Trump’s sprint to take control of th...,https://www.wsj.com/politics/policy/trump-fed-...,128
360,2025-09-04-18-19-18 +0000,bbc,Trump's Fed pick says he will keep White House...,https://www.bbc.com/news/articles/c5yq3z2xd01o...,110
222,2025-09-05-02-44-00 +0000,wsj,President Trump was within his authority when ...,https://www.wsj.com/politics/national-security...,109
193,2025-09-05-06-52-10 +0000,nypost,"Trump’s ‘pro-business, pro-innovation’ agenda ...",https://nypost.com/2025/09/05/us-news/trumps-p...,108


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
243,130,2025-09-05-00-19-00 +0000,wsj,President Trump is expected to sign an executi...,https://www.wsj.com/politics/national-security...
39,65,2025-09-05-15-41-00 +0000,wsj,The Florida immigration detention facility kno...,https://www.wsj.com/politics/policy/florida-al...
94,61,2025-09-05-13-12-22 +0000,nyt,Elon Musk Could Become First Trillionaire Unde...,https://www.nytimes.com/2025/09/05/business/el...
237,61,2025-09-05-01-00-00 +0000,wsj,President Trump’s sprint to take control of th...,https://www.wsj.com/politics/policy/trump-fed-...
41,50,2025-09-05-15-40-00 +0000,wsj,Putin Says Russia Will Strike Any Western Troo...,https://www.wsj.com/world/russia/putin-says-ru...
193,48,2025-09-05-06-52-10 +0000,nypost,"Trump’s ‘pro-business, pro-innovation’ agenda ...",https://nypost.com/2025/09/05/us-news/trumps-p...
190,48,2025-09-05-07-53-33 +0000,wsj,Stock Market Today: Weak Jobs Data Boosts Rate...,https://www.wsj.com/livecoverage/jobs-report-a...
302,47,2025-09-04-21-25-00 +0000,wsj,Health Secretary Robert F. Kennedy Jr. was gri...,https://www.wsj.com/politics/policy/rfk-jr-cdc...
50,45,2025-09-05-15-21-27 +0000,nyt,Israel Steps Up Attacks on Gaza City Ahead of ...,https://www.nytimes.com/2025/09/05/world/middl...
352,39,2025-09-04-18-50-00 +0000,wsj,The Justice Department opened a criminal inves...,https://www.wsj.com/politics/policy/lisa-cook-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
